# <font color = 'red'> Pré Processamento e Engenharia de Atributos</font>

### <font color = 'red'> Foco: </font>

- Pré processar o texto para a etapa de análise de sentimentos

- Remoção de stopwords e outros, para verificar ocorrência

In [58]:
# Imports
import os
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize

import spacy



import warnings
warnings.filterwarnings('ignore')


In [65]:
# Visualizando Cj dados
dados = pd.read_csv('/Projetos Pessoais/DataScience/PLN_Text_Analysis/data/data_scraping_bruto.csv',
                       sep =',')

In [66]:
dados.head()

,Empresa,Relato,Avaliacao
0,Serasa Experian,C. A legislação consumerista (à luz da qual de...,Nota 5
1,Serasa Experian,Solicito a baixa dos meus dados junto ao SERAS...,Nota 1
2,Serasa Experian,Eu solicito a limpeza em seus cadastros,Nota 5
3,Latam Airlines (Tam),"Prezados, apesar de ser cliente da categoria g...",Nota 5
4,Serasa Experian,Em 04/11/2024 verifiquei o extrato da conta co...,Nota 1


In [67]:
instituicoes_financeiras = ['Nubank',
              'Banco do Brasil',
              'Caixa Econômica Federal',
              'Banco Santander',
              'Banco Itaú Unibanco',
              'Mercado Pago',
              'Banco Inter (Banco Intermedium)',
              'Banco Bradesco',
              'C6 Bank']

In [68]:
# Selecionando apenas instituições financeiras

dados_financeiros = dados[dados['Empresa'].isin(instituicoes_financeiras)]

In [69]:
#Verificando ocorrências

dados_financeiros.Empresa.value_counts().reset_index(name='Quantidade Relatos')

,Empresa,Quantidade Relatos
0,Nubank,484
1,Banco do Brasil,365
2,Caixa Econômica Federal,242
3,Banco Santander,212
4,Banco Itaú Unibanco,142
5,Mercado Pago,139
6,Banco Bradesco,137
7,Banco Inter (Banco Intermedium),127
8,C6 Bank,72


In [70]:
dados_financeiros.head(5)

,Empresa,Relato,Avaliacao
23,Caixa Econômica Federal,Não consigo crédito pra fazer um empréstimo me...,Nota 3
41,Nubank,ja houve o pagamento da negociacao,Nota 2
45,Nubank,Exclusao do nome Retirar negativacao do nome s...,Nota 1
49,C6 Bank,"Sou cliente do C6 Bank há tempos, sempre mante...",Nota 1
52,Nubank,Gostaria de reclamar sobre essa dívida quer Es...,Nota 5


In [71]:
dados_financeiros['Avaliacao'] = dados_financeiros['Avaliacao'].str.extract(r'(\d+)')

In [72]:
dados_financeiros.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1920 entries, 23 to 10929
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Empresa    1920 non-null   object
 1   Relato     1920 non-null   object
 2   Avaliacao  1920 non-null   object
dtypes: object(3)
memory usage: 60.0+ KB


### <font color='green'> Remoção de pontuações e deixando o texto em caixa baixa

In [73]:
def processa_sentimento(text):
 # Minuscula
 text = text.lower()

 # retirando pontuações, exceto "."
 text = re.sub(r'[^\w\s.]', '', text)

 return text

In [74]:
dados_financeiros['Relato'] = dados_financeiros['Relato'].apply(processa_sentimento)

In [75]:
dados_financeiros.sample(5)

,Empresa,Relato,Avaliacao
4344,Caixa Econômica Federal,peço o cancelamento desse serviço por não muit...,5
4977,Banco Inter (Banco Intermedium),protocolo anterior nenhuma das reclamações for...,1
494,Nubank,estou com o nome limpo e não consigo fazer car...,1
2006,Caixa Econômica Federal,meu contrato do empréstimo consignado da caixa...,5
9005,Mercado Pago,efetuado consulta no registrato do banco centr...,5


In [76]:
dados_financeiros.to_csv('/Projetos Pessoais/DataScience/PLN_Text_Analysis/data/dados_financeiros_sentimentos.csv',
                       sep =',', index = False, encoding = 'utf-8')


### <font color='green'> Pré processamento para frequência - TF IDF e Collocations

- Tokennizando os textos

In [77]:
word_tokenize("Data Science Academy 2024")

['Data', 'Science', 'Academy', '2024']

In [79]:
# Carregar modelo em português
lematizacao = spacy.load("pt_core_news_sm") 

# Aplicar lematização em cada relato
dados_financeiros['Relato_Lematizado'] = dados_financeiros['Relato'].apply(lambda tokens: [token.lemma_ for token in lematizacao(" ".join(tokens))])

In [81]:
nlp = spacy.load("pt_core_news_sm")

# Função para lematizar uma frase completa
def lematizar_frase(frase):
    doc = nlp(frase)
    return " ".join([token.lemma_ for token in doc])

dados_financeiros['Relato_Lematizado'] = dados_financeiros['Relato'].apply(lematizar_frase)

# Exibir o DataFrame resultante
print(dados_financeiros[['Empresa', 'Relato', 'Avaliacao', 'Relato_Lematizado']])


                       Empresa  \
23     Caixa Econômica Federal   
41                      Nubank   
45                      Nubank   
49                     C6 Bank   
52                      Nubank   
...                        ...   
10910      Banco Itaú Unibanco   
10912      Banco Itaú Unibanco   
10923                  C6 Bank   
10924           Banco Bradesco   
10929  Caixa Econômica Federal   

                                                  Relato Avaliacao  \
23     não consigo crédito pra fazer um empréstimo me...         3   
41                    ja houve o pagamento da negociacao         2   
45     exclusao do nome retirar negativacao do nome s...         1   
49     sou cliente do c6 bank há tempos sempre manten...         1   
52     gostaria de reclamar sobre essa dívida quer es...         5   
...                                                  ...       ...   
10910  solicitando a retirada da restrição de apontam...         4   
10912  estou com dificuldade pa

In [82]:
dados_financeiros

,Empresa,Relato,Avaliacao,Relato_Lematizado
23,Caixa Econômica Federal,não consigo crédito pra fazer um empréstimo me...,3,não consigir crédito pra fazer um empréstimo m...
41,Nubank,ja houve o pagamento da negociacao,2,ja haver o pagamento de o negociacao
45,Nubank,exclusao do nome retirar negativacao do nome s...,1,exclusao de o nome retirar negativacao de o no...
49,C6 Bank,sou cliente do c6 bank há tempos sempre manten...,1,ser cliente de o c6 bank haver tempo sempre ma...
52,Nubank,gostaria de reclamar sobre essa dívida quer es...,5,gostar de reclamar sobre esse dívida querer es...
...,...,...,...,...
10910,Banco Itaú Unibanco,solicitando a retirada da restrição de apontam...,4,solicitar o retirada de o restrição de apontam...
10912,Banco Itaú Unibanco,estou com dificuldade para redefinir a senha d...,1,estar com dificuldade para redefinir o senha d...
10923,C6 Bank,gostaria que tirassem o apontamento no registr...,5,gostar que tirar o apontamento em o registrato...
10924,Banco Bradesco,sou cliente do banco bradesco e possuo um cheq...,1,ser cliente de o banco bradesco e possuor um c...


In [84]:
dados_financeiros['Len_Relato_Lematizado'] = dados_financeiros['Relato_Lematizado'].apply(len)

In [91]:
len(dados_financeiros['Relato'].iloc[0])

195

In [90]:
len(dados_financeiros['Relato_Lematizado'].iloc[0])

199

In [80]:
dados_financeiros.head()

,Empresa,Relato,Avaliacao,Relato_Lematizado
23,Caixa Econômica Federal,não consigo crédito pra fazer um empréstimo me...,3,"[n, ã, o, , c, o, n, s, i, g, o, , c, elr,..."
41,Nubank,ja houve o pagamento da negociacao,2,"[j, a, , h, o, u, v, e, , o, , p, a, g, ..."
45,Nubank,exclusao do nome retirar negativacao do nome s...,1,"[e, x, c, l, u, s, a, o, , d, o, , n, o, m..."
49,C6 Bank,sou cliente do c6 bank há tempos sempre manten...,1,"[s, o, u, , c, l, i, e, n, t, e, , d, o, ..."
52,Nubank,gostaria de reclamar sobre essa dívida quer es...,5,"[g, o, s, tr, a, r, i, a, , d, e, , r, e, ..."


In [48]:
tokenizer = WordPunctTokenizer()

def tokeniza(texto):
 token = word_tokenize(texto)
 return token

dados_financeiros['Relato'] = dados_financeiros['Relato'].map(lambda x: tokeniza(x))

In [49]:
dados_financeiros.head()

,Empresa,Relato,Avaliacao
23,Caixa Econômica Federal,"[não, consigo, crédito, pra, fazer, um, emprés...",3
41,Nubank,"[ja, houve, o, pagamento, da, negociacao]",2
45,Nubank,"[exclusao, do, nome, retirar, negativacao, do,...",1
49,C6 Bank,"[sou, cliente, do, c6, bank, há, tempos, sempr...",1
52,Nubank,"[gostaria, de, reclamar, sobre, essa, dívida, ...",5


- Tratamento de StopWords

In [50]:
# Stopwords em portugues
stopwords_PT = set(stopwords.words("portuguese"))

In [54]:
# Função para verificar se está com Stopwords no relato dos clientes - Utilizando list comprehension

def veriricaStopWord(texto):
 texto_filtrado = [palavra for palavra in texto if palavra not in stopwords_PT]
 return texto_filtrado

In [55]:
dados_financeiros['Relato'] = dados_financeiros['Relato'].map(lambda x: veriricaStopWord(x))

In [56]:
dados_financeiros.head()

,Empresa,Relato,Avaliacao
23,Caixa Econômica Federal,"[consigo, crédito, pra, fazer, empréstimo, nom...",3
41,Nubank,"[ja, pagamento, negociacao]",2
45,Nubank,"[exclusao, nome, retirar, negativacao, nome, s...",1
49,C6 Bank,"[cliente, c6, bank, tempos, sempre, mantendo, ...",1
52,Nubank,"[gostaria, reclamar, sobre, dívida, quer, cons...",5


- Etapa de Lematização

In [59]:
# Carregar modelo em português
lematizacao = spacy.load("pt_core_news_sm") 

# Aplicar lematização em cada relato
dados_financeiros['Relato_Lematizado'] = dados_financeiros['Relato'].apply(lambda tokens: [token.lemma_ for token in lematizacao(" ".join(tokens))])

In [60]:
dados_financeiros.head()

,Empresa,Relato,Avaliacao,Relato_Lematizado
23,Caixa Econômica Federal,"[consigo, crédito, pra, fazer, empréstimo, nom...",3,"[com si, crédito, pra, fazer, empréstimo, nome..."
41,Nubank,"[ja, pagamento, negociacao]",2,"[ja, pagamento, negociacao]"
45,Nubank,"[exclusao, nome, retirar, negativacao, nome, s...",1,"[exclusao, nome, retirar, negativacao, nome, s..."
49,C6 Bank,"[cliente, c6, bank, tempos, sempre, mantendo, ...",1,"[cliente, C6, bank, tempo, sempre, manter, rel..."
52,Nubank,"[gostaria, reclamar, sobre, dívida, quer, cons...",5,"[gostar, reclamar, sobre, dívida, querer, cons..."


In [61]:
import torch
from transformers import AutoTokenizer, AutoModel

dados_financeiros['Relato'] = dados_financeiros['Relato'].apply(lambda x: " ".join(x))

# Carregar o modelo e tokenizer
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = AutoModel.from_pretrained("neuralmind/bert-base-portuguese-cased")

def get_embeddings(text):
    # Tokenizar o texto
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Extrair embeddings (usamos a camada [CLS] para representar a frase)
    embeddings = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return embeddings

# Aplicar a função ao dataframe e criar a nova coluna com os embeddings
dados_financeiros['Relato_Lematizado'] = dados_financeiros['Relato'].apply(get_embeddings)

# Exibir o dataframe resultante
print(dados_financeiros[['Empresa', 'Relato', 'Avaliacao', 'Relato_Lematizado']])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


                       Empresa  \
23     Caixa Econômica Federal   
41                      Nubank   
45                      Nubank   
49                     C6 Bank   
52                      Nubank   
...                        ...   
10910      Banco Itaú Unibanco   
10912      Banco Itaú Unibanco   
10923                  C6 Bank   
10924           Banco Bradesco   
10929  Caixa Econômica Federal   

                                                  Relato Avaliacao  \
23     consigo crédito pra fazer empréstimo nome enco...         3   
41                               ja pagamento negociacao         2   
45      exclusao nome retirar negativacao nome spcserasa         1   
49     cliente c6 bank tempos sempre mantendo relacio...         1   
52     gostaria reclamar sobre dívida quer constando ...         5   
...                                                  ...       ...   
10910  solicitando retirada restrição apontamento ind...         4   
10912  dificuldade redefinir se

In [63]:
nlp = spacy.load("pt_core_news_sm")

def lemmatize_text(text):
    doc = nlp(text)
    # Extrair a forma lematizada de cada token e juntar em uma string
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

# Aplicar a função ao dataframe
dados_financeiros['Relato_Lematizado'] = dados_financeiros['Relato'].apply(lemmatize_text)

# Exibir o dataframe resultante
print(dados_financeiros[['Empresa', 'Relato', 'Avaliacao', 'Relato_Lematizado']])

                       Empresa  \
23     Caixa Econômica Federal   
41                      Nubank   
45                      Nubank   
49                     C6 Bank   
52                      Nubank   
...                        ...   
10910      Banco Itaú Unibanco   
10912      Banco Itaú Unibanco   
10923                  C6 Bank   
10924           Banco Bradesco   
10929  Caixa Econômica Federal   

                                                  Relato Avaliacao  \
23     consigo crédito pra fazer empréstimo nome enco...         3   
41                               ja pagamento negociacao         2   
45      exclusao nome retirar negativacao nome spcserasa         1   
49     cliente c6 bank tempos sempre mantendo relacio...         1   
52     gostaria reclamar sobre dívida quer constando ...         5   
...                                                  ...       ...   
10910  solicitando retirada restrição apontamento ind...         4   
10912  dificuldade redefinir se

- Retirando NoAscii

,Empresa,Relato,Avaliacao,Relato_Lematizado
23,Caixa Econômica Federal,consigo crédito pra fazer empréstimo nome enco...,3,com si crédito pra fazer empréstimo nome encon...
41,Nubank,ja pagamento negociacao,2,ja pagamento negociacao
45,Nubank,exclusao nome retirar negativacao nome spcserasa,1,exclusao nome retirar negativacao nome spcserasa
49,C6 Bank,cliente c6 bank tempos sempre mantendo relacio...,1,cliente C6 bank tempo sempre manter relacionam...
52,Nubank,gostaria reclamar sobre dívida quer constando ...,5,gostar reclamar sobre dívida querer constar no...


In [20]:
def removeNoascii(s):
 return "".join(i for i in s if ord(i) < 128)

# Remove características non-ascii
dados_financeiros['Relato'] = dados_financeiros['Relato'].map(lambda x: removeNoascii(x))

In [22]:
dados_financeiros.Relato.head()

23    no consigo crdito pra fazer um emprstimo meu n...
41                   ja houve o pagamento da negociacao
45    exclusao do nome retirar negativacao do nome s...
49    sou cliente do c6 bank h tempos sempre mantend...
52    gostaria de reclamar sobre essa dvida quer est...
Name: Relato, dtype: object

- Retirando StopWord

In [23]:
stopwords_PT = set(stopwords.words("portuguese"))

In [ ]:
stopwords_PT

{'a',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'estamos',
 'estar',
 'estas',
 'estava',
 'estavam',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéramos',
 'estivéssemos',
 'estou',
 'está',
 'estávamos',
 'estão',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fui',
 'fôramos',
 'fôssemos',
 'haja',
 'hajam',
 'hajamos',
 'havemos',
 'haver',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houveram',
 'houverei',
 'houverem',
 'houveremos',
 'houveria',
 'houveriam',
 'houvermos',
 'houverá',
 'houverão',
 'houverí